In [1]:
! pip install biopython

In [2]:
# ! pip install biopython
from Bio import pairwise2
from Bio.PDB import PDBList, MMCIFParser

# Given DNA sequences
dna_seq1 = "AGGTGAGAGGCCGGAACCGGCTTTTCATAT"
dna_seq2 = ""

# Function to fetch PDB file and extract sequences
def fetch_pdb_sequences(pdb_id):
    pdb_downloader = PDBList()
    pdb_file = pdb_downloader.retrieve_pdb_file(pdb_id, file_format='mmCif')
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure(pdb_id, pdb_file)

    sequences = {}
    for model in structure:
        for chain in model:
            if chain.id not in sequences:
                sequences[chain.id] = {"protein": "", "dna": ""}
            for residue in chain:
                if residue.id[0] == " " and residue.resname in ['DA', 'DT', 'DG', 'DC']:  # Identifying DNA residues
                    sequences[chain.id]["dna"] += residue.resname[1]
    return sequences

# Fetch sequences from PDB ID 5KE6
sequences_5ke6 = fetch_pdb_sequences("2ER8")

# Function to check sequence matches
def check_sequence_matches(given_sequence, pdb_sequences):
    results = []
    for chain_id, seq_data in pdb_sequences.items():
        dna_seq = seq_data["dna"]
        if dna_seq:
            alignments = pairwise2.align.localms(dna_seq, given_sequence, 2, -1, -0.5, -0.1)
            for alignment in alignments:
                aligned_pdb_seq, aligned_given_seq, score, start, end = alignment
                matches = sum(1 for a, b in zip(aligned_pdb_seq, aligned_given_seq) if a == b)
                identity = matches / len(given_sequence) * 100
                results.append({
                    "chain": chain_id,
                    "identity": identity,
                    "similarity": identity,  # For DNA, identity and similarity are typically the same
                    "pdb_aligned": aligned_pdb_seq,
                    "given_aligned": aligned_given_seq
                })
    return results

# Check matches for both given sequences
matches_seq1 = check_sequence_matches(dna_seq1, sequences_5ke6)
matches_seq2 = check_sequence_matches(dna_seq2, sequences_5ke6)

# Display results in a column format
def display_results(matches, seq_label):
    print(f"Results for {seq_label}:")
    print(f"{'Chain':<10}{'Identity (%)':<15}{'Similarity (%)':<15}{'PDB Sequence Aligned':<30}{'Given Sequence Aligned':<30}")
    print("-" * 100)
    for match in matches:
        print(f"{match['chain']:<10}{match['identity']:<15.2f}{match['similarity']:<15.2f}{match['pdb_aligned']:<30}{match['given_aligned']:<30}")
    print("\n")

display_results(matches_seq1, "DNA SEQ1")
display_results(matches_seq2, "DNA SEQ2")


/home/codespace/.python/current/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


Results for DNA SEQ1:
Chain     Identity (%)   Similarity (%) PDB Sequence Aligned          Given Sequence Aligned        
----------------------------------------------------------------------------------------------------
E         30.00          30.00          ---------CCCGGTA-CCGGG---------AGGTGAGAGGCCGG-AACCGGCTTTTCATAT
E         30.00          30.00          ---------CCCGGT-ACCGGG---------AGGTGAGAGGCCGG-AACCGGCTTTTCATAT
E         30.00          30.00          ---------CCCGGTACCGGG---------AGGTGAGAGGCCGGAACCGGCTTTTCATAT
F         30.00          30.00          ---------CCCGGTA-CCGGG---------AGGTGAGAGGCCGG-AACCGGCTTTTCATAT
F         30.00          30.00          ---------CCCGGT-ACCGGG---------AGGTGAGAGGCCGG-AACCGGCTTTTCATAT
F         30.00          30.00          ---------CCCGGTACCGGG---------AGGTGAGAGGCCGGAACCGGCTTTTCATAT
G         30.00          30.00          ---------CCCGGTA-CCGGG---------AGGTGAGAGGCCGG-AACCGGCTTTTCATAT
G         30.00          30.00          ---------CCCGGT-ACC